In [1]:
from grid import Grid

g = Grid()
g.print()

-------------------------------
| 0,0 | 0,1 | 0,2 | 0,3 | 0,4 | 
-------------------------------
| 1,0 | 1,1 | 1,2 | 1,3 | 1,4 | 
-------------------------------
| 2,0 | 2,1 | 2,2 | 2,3 | 2,4 | 
-------------------------------
| 3,0 | 3,1 | 3,2 | 3,3 | 3,4 | 
-------------------------------
| 4,0 | 4,1 | 4,2 | 4,3 | 4,4 | 
-------------------------------


In [2]:
g.transition((4,0), g.west)

((4, 0), -1)

In [3]:
g.transition((0,1), g.north)

((4, 1), 10)

In [8]:
pos = (0,0)
custom_grid = g.grid.copy()
custom_grid[pos] = 0.1
g.print(custom_grid)
print()

custom_grid = g.grid.copy()
s,r =  g.transition(pos, g.east)
custom_grid[s] = r+0.1
print(s)
g.print(custom_grid)

-------------------------------
| 0.1 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------

(4, 1)
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
-------------------------------
| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 
--------------------------------
| 0.0 | 10.1 | 0.0 | 0.0 | 0.0 | 
--------------------------------


In [17]:
import numpy as np
def v(state=None, gamma=0.9, p=0.25):
    ACTIONS = [g.north, g.south, g.east, g.west]

    A = -1 * np.eye(g.size[0] * g.size[1])
    b = np.zeros(g.size[0] * g.size[1])

    for i in range(g.size[0]):
        for j in range(g.size[1]):
            s = (i, j)  # for each state s
            index_s = np.ravel_multi_index(s, g.size)
            for a in ACTIONS:
                s_, r = g.transition(s, a)
                index_s_ = np.ravel_multi_index(s_, g.size)

                A[index_s, index_s_] += p * gamma
                b[index_s] -= p * r
    v_matrix = np.round(np.linalg.solve(A, b).reshape(*g.size), decimals=2)
    if state is None:
        return v_matrix
    else:
        return v_matrix[state]

In [18]:
print(v())

[[ 3.89  5.01  5.07  3.46  1.93]
 [ 1.88  3.44  2.62  2.27  0.83]
 [ 0.25  0.96  0.87  0.55 -0.24]
 [-0.86 -0.32 -0.24 -0.48 -1.08]
 [-1.78 -1.26 -1.15 -1.35 -1.9 ]]


In [19]:
WORLD_SIZE = 5
A_POS = [0, 1]
A_PRIME_POS = [4, 1]
B_POS = [0, 3]
B_PRIME_POS = [2, 3]
DISCOUNT = 0.9

# left, up, right, down
ACTIONS = [np.array([0, -1]),
           np.array([-1, 0]),
           np.array([0, 1]),
           np.array([1, 0])]
ACTIONS_FIGS=[ '←', '↑', '→', '↓']


ACTION_PROB = 0.25


def step(state, action):
    if state == A_POS:
        return A_PRIME_POS, 10
    if state == B_POS:
        return B_PRIME_POS, 5

    next_state = (np.array(state) + action).tolist()
    x, y = next_state
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        reward = -1.0
        next_state = state
    else:
        reward = 0
    return next_state, reward

A = -1 * np.eye(WORLD_SIZE * WORLD_SIZE)
b = np.zeros(WORLD_SIZE * WORLD_SIZE)
for i in range(WORLD_SIZE):
    for j in range(WORLD_SIZE):
        s = [i, j]  # current state
        index_s = np.ravel_multi_index(s, (WORLD_SIZE, WORLD_SIZE))
        for a in ACTIONS:
            s_, r = step(s, a)
            index_s_ = np.ravel_multi_index(s_, (WORLD_SIZE, WORLD_SIZE))

            A[index_s, index_s_] += ACTION_PROB * DISCOUNT
            b[index_s] -= ACTION_PROB * r

x = np.linalg.solve(A, b)
np.round(x.reshape(WORLD_SIZE, WORLD_SIZE), decimals=2)

array([[ 3.31,  8.79,  4.43,  5.32,  1.49],
       [ 1.52,  2.99,  2.25,  1.91,  0.55],
       [ 0.05,  0.74,  0.67,  0.36, -0.4 ],
       [-0.97, -0.44, -0.35, -0.59, -1.18],
       [-1.86, -1.35, -1.23, -1.42, -1.98]])

In [ ]:
for i in range(WORLD_SIZE):
    for j in range(WORLD_SIZE):
        s = [i, j] 
        for a in ACTIONS:
            s_, r_ = step(s, a)
            s_2, r_2 = g.transition(tuple(s), tuple(a))

            if (s_ != s_2) or (r_ != r_2):
                print(s)
                print(s_, r_)
                print(s_2, r_2)
